# Mise en base de l'arborescence

## Documentation

### Liste des tables

A chaque type de fichiers mxl correspond une table :
* article : fichiers LEGIARTIxxxxxxxxxxxx.xml
* section_ta : fichiers LEGISCTAxxxxxxxxxxxx.xml
* struct : fichiers struct/LEGITEXTxxxxxxxxxxxx.xml
* version : fichiers version/LEGITEXTxxxxxxxxxxxx.xml

### Liste des champs

#### categorie

* code_en_vigueur_LEGI
* code_non_vigueur_LEGI
* TNC_en_vigueur_JORF
* TNC_en_vigueur_LEGI
* TNC_non_vigueur_JORF 
* TNC_non_vigueur_LEGI

#### cid

Correspond au 2e niveau dans l'arborescence plate.

#### id_

Correspond au nom des fichiers dans l'arborescence plate. Un underscore est ajouté à la fin car le mot "id" est un mot réservé pour certains langages de programmation, dont python.

## Mode opératoire

* Créer l'utilisateur et la base de donnée
```
# sudo -i -u postgres
$ createuser --interactive
$ createdb loi
$ psql
# grant all privileges on database loi to loi;
# alter user loi with password 'baba';
```

* Ajouter la ligne suivante à /etc/postgresql/9.6/main/pg_hba.conf : `local loi loi md5`
* Redémarrer postgresql : `service postgresql restart`
* Pour se connecter avec psql :
```
$ psql -U loi loi
```

* Créer les tables :
```
create table article (categorie char(21), cid char(21), id_ char(21));
create table section_ta (categorie char(21), cid char(21), id_ char(21));
create table version (categorie char(21), cid char(21), id_ char(21));
create table struct (categorie char(21), cid char(21), id_ char(21));
```

In [1]:
import os
import re

import psycopg2

In [2]:
connection = psycopg2.connect(dbname='loi', user='loi', password='baba')
curseur = connection.cursor()

In [3]:
racine_source = '/home/michel/legi_plat/'

In [11]:
def trouve_enfants(dossier):
    enfants = os.listdir(dossier)
    est_fichier_liste = [os.path.isfile(os.path.join(dossier, f)) for f in enfants]
    enfants_fichier = [enfants[i] for i in range(len(enfants)) if est_fichier_liste[i]]
    enfants_dossier = [enfants[i] for i in range(len(enfants)) if not est_fichier_liste[i]]
    
    return enfants_fichier, enfants_dossier

In [12]:
def traite_struct(racine, categorie, cid):
    dossier = os.path.join(racine, categorie, cid, 'struct')
    enfants_fichier, enfants_dossier = trouve_enfants(dossier)
    
    assert len(enfants_dossier) == 0
    
    nom_fichier_attendu = '^LEGITEXT\d{12}\.xml$'
    for enfant_fichier in enfants_fichier:
        assert re.match(nom_fichier_attendu, enfant_fichier)
        
        id_ = enfant_fichier[:20]
        curseur.execute("insert into struct (categorie, cid, id_) values (%s, %s, %s)", (categorie, cid, id_,))
        connection.commit()

In [13]:
def traite_version(racine, categorie, cid):
    dossier = os.path.join(racine, categorie, cid, 'version')
    enfants_fichier, enfants_dossier = trouve_enfants(dossier)
    
    assert len(enfants_dossier) == 0
    
    nom_fichier_attendu = '^LEGITEXT\d{12}\.xml$'
    for enfant_fichier in enfants_fichier:
        assert re.match(nom_fichier_attendu, enfant_fichier)
        
        id_ = enfant_fichier[:20]
        curseur.execute("insert into version (categorie, cid, id_) values (%s, %s, %s)", (categorie, cid, id_,))
        connection.commit()

In [14]:
def traite_cid(racine, categorie, cid):
    dossier = os.path.join(racine, categorie, cid)
    enfants_fichier, enfants_dossier = trouve_enfants(dossier)
    
    assert set(enfants_dossier) == set([
        'struct',
        'version',
    ])
    traite_struct(racine, categorie, cid)
    traite_version(racine, categorie, cid)
    
    nom_fichier_article = '^LEGIARTI\d{12}\.xml$'
    nom_fichier_section_ta = '^LEGISCTA\d{12}\.xml$'
    for enfant_fichier in enfants_fichier:
        id_ = enfant_fichier[:20]
        if re.match(nom_fichier_article, enfant_fichier):
            curseur.execute("insert into article (categorie, cid, id_) values (%s, %s, %s)", (categorie, cid, id_,))
            connection.commit()
        elif re.match(nom_fichier_section_ta, enfant_fichier):
            curseur.execute("insert into section_ta (categorie, cid, id_) values (%s, %s, %s)", (categorie, cid, id_,))
            connection.commit()
        else:
            raise ValueError(cid)


In [15]:
def traite_categorie(racine, categorie):
    dossier = os.path.join(racine, categorie)
    enfants_fichier, enfants_dossier = trouve_enfants(dossier)
    
    assert len(enfants_fichier) == 0
    
    nom_dossier_attendu = '^' + categorie[-4:] + 'TEXT\d{12}$'
    for enfant_dossier in enfants_dossier:
        assert re.match(nom_dossier_attendu, enfant_dossier)
        
        traite_cid(racine, categorie, enfant_dossier)
    

In [16]:
def traite_tout(racine):
    enfants_fichier, enfants_dossier = trouve_enfants(racine)
    
    assert len(enfants_fichier) == 0
    assert set(enfants_dossier) == set([
            'code_en_vigueur_LEGI',
            'code_non_vigueur_LEGI',
            'TNC_en_vigueur_JORF',
            'TNC_en_vigueur_LEGI',
            'TNC_non_vigueur_JORF',
            'TNC_non_vigueur_LEGI',
        ])
        
    for enfant_dossier in enfants_dossier:
        traite_categorie(racine, enfant_dossier)

In [ ]:
traite_tout(racine_source)

In [ ]:


while True:
    curseur.execute("select * from article where (status is null or status != %s) and (mod(numero_ligne, %s) = %s) limit 1;", (version_courante, nb_proc, id_proc))
    resultat_requete = curseur.fetchone()
    
    if not resultat_requete:
        break
        
    nom_fichier = resultat_requete[0]
        
    with open(repertoire_racine + 'article/' + nom_fichier) as f:
        contenu = f.read()
    
    valeurs = parse_contenu(contenu)
    
    valeurs['status'] = version_courante
    valeurs['nom_fichier'] = nom_fichier

    curseur.execute("""
        update article set
            ID = %(ID)s,
            ID_ELI = %(ID_ELI)s,
            ID_ELI_ALIAS = %(ID_ELI_ALIAS)s,
            ANCIEN_ID = %(ANCIEN_ID)s,
            MCS_ART = %(MCS_ART)s,
            NUM = %(NUM)s,
            TYPE = %(TYPE)s,
            URL = %(URL)s,
            TEXTE_cid = %(TEXTE_cid)s,
            BLOC_TEXTUEL = %(BLOC_TEXTUEL)s,
            DATE_DEBUT = %(DATE_DEBUT)s,
            DATE_FIN = %(DATE_FIN)s,
            status = %(status)s
        where nom_fichier = %(nom_fichier)s;
        """, valeurs)
    connection.commit()



In [ ]:

curseur.close()
connection.close()